# Instruction

**Run each cell one by one**

#Load_data

In [1]:
from keras.models import model_from_json
import numpy as np
import scipy.io as sio
import glob as glob
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
import keras.backend as K
from tensorflow.keras.layers import Layer
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Activation, add, Dense, Flatten, Dropout, Multiply, Embedding, Lambda
from keras.layers import Conv2D, MaxPooling2D,PReLU,concatenate
from keras import backend as K
import numpy as np
import sys
from keras.callbacks import *
from tensorflow.keras.optimizers import SGD, Adam
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from keras.losses import categorical_crossentropy
import numpy as np
from keras.models import Model
from keras.layers.core import Dense, Activation
from keras.layers.convolutional import Conv1D,Conv2D, AveragePooling2D,SeparableConv2D
from tensorflow.keras.layers import BatchNormalization
from keras.layers import Dropout, Add, Lambda,DepthwiseConv2D,Input, Permute,Flatten
from tensorflow.keras.constraints import max_norm
import random
import keras
from sklearn.decomposition import PCA

In [2]:
from google.colab import drive
drive.mount('/content/drive')

!pip install mne



import numpy as np
import scipy.io as sio
import mne
def indexget(total,select):
  return [i for i, e in enumerate(total) if e in select]



def inputmat(fp1,fp2,fp3):
    """load .mat file and return m as a dict"""
    mat = sio.loadmat(fp1, squeeze_me=True)
    label_mat=sio.loadmat(fp2, squeeze_me=True)
    m = {}  # create a dict

    # Numpy array of size channel_num * points.
    select_channel=fp3
    channelindex= indexget(mat['nfo']['clab'][True][0],select_channel)
    m['ch_names'] = mat['nfo']['clab'][True][0][channelindex]
    


    m['data'] = mat['cnt'].T[channelindex]
    m['freq'] = mat['nfo']['fs'][True][0]  # Sampling frequency

    # channel names are necessary information for creating a rawArray.


    # Position of channels
    m['electrode_x'] = mat['nfo']['xpos'][True][0]
    m['electrode_y'] = mat['nfo']['ypos'][True][0]

    # find trials and its data
    m['cue'] = mat['mrk']['pos'][True][0]  # time of cue
    m['labels'] = label_mat['true_y']  
    # m['labels'] = np.nan_to_num(mat['mrk']['y'][True][0]).astype(int)  # convert NaN to 0
    m['test_idx'] = len(label_mat['test_idx'])
    m['n_trials'] = len(m['labels'])  # Number of the total useful trials
    return m


def creatEventsArray(fp1,fp2,fp3):
    """Create events array. The second column default to zero."""
    m = inputmat(fp1,fp2,fp3)
    events = np.zeros((m['n_trials'], 3), int)
    events[:, 0] = m['cue'][:m['n_trials']]  # The first column is the sample number of the event.
    events[:, 2] = m['labels'][:m['n_trials']]  # The third column is the new event value.
    return events, m['labels']


def creatRawArray(fp1,fp2,fp3):
    """Create a mne.io.RawArray object, data: array, shape (n_channels, n_times)"""
    m = inputmat(fp1,fp2,fp3)
    ch_names = m['ch_names'].tolist()
    info = mne.create_info(ch_names, m['freq'], 'eeg')  # Create info for raw
    raw = mne.io.RawArray(m['data'], info, first_samp=0, copy='auto', verbose=None)
    return raw

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.5 MB 5.6 MB/s 


In [3]:
print(__doc__)

fp = {
    'aa': '/content/drive/MyDrive/fewshotonlineBCI/CompetitionIII_IVa/data_set_IVa_aa.mat',
    'al': '/content/drive/MyDrive/fewshotonlineBCI/CompetitionIII_IVa/data_set_IVa_al.mat',
    'av': '/content/drive/MyDrive/fewshotonlineBCI/CompetitionIII_IVa/data_set_IVa_av.mat',
    'aw': '/content/drive/MyDrive/fewshotonlineBCI/CompetitionIII_IVa/data_set_IVa_aw.mat',
    'ay': '/content/drive/MyDrive/fewshotonlineBCI/CompetitionIII_IVa/data_set_IVa_ay.mat',
}


fplabel={
    'aa': '/content/drive/MyDrive/fewshotonlineBCI/CompetitionIII_IVa/true_labels_aa.mat',
    'al': '/content/drive/MyDrive/fewshotonlineBCI/CompetitionIII_IVa/true_labels_al.mat',
    'av': '/content/drive/MyDrive/fewshotonlineBCI/CompetitionIII_IVa/true_labels_av.mat',
    'aw': '/content/drive/MyDrive/fewshotonlineBCI/CompetitionIII_IVa/true_labels_aw.mat',
    'ay': '/content/drive/MyDrive/fewshotonlineBCI/CompetitionIII_IVa/true_labels_ay.mat',

}

channel_set=['C1',  'C3',  'Cz',  'C2',  'C4',  'CFC3',  'CFC4','CFC5',  'CFC6',  'CCP3', 
             'CCP4',  'CCP5',  'CCP6',  'T7',  'T8',  'P3',  'Pz',  'P4']
pick_chan = {
    'aa':  channel_set,
    'al':  channel_set,
    'av':  channel_set,
    'aw':  channel_set,
    'ay':  channel_set,

}

low_freq, high_freq = 7., 30.
tmin, tmax = 0., 3.5
# event_id
event_id = {'right': 1, 'foot': 2}

Automatically created module for IPython interactive environment


# Processing the data

In [4]:
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.pipeline import Pipeline

from mne.decoding import CSP
from mne.channels import read_layout



def cal_acc(pred,real):
  return [1 if pred[j]==real[j] else 0 for j in range(len(real))]

def getdata_label(a,b,c):
  mat_dic=inputmat(a,b,c)
  raw = creatRawArray(a,b,c)
  events, labels = creatEventsArray(a,b,c)


    # Apply band-pass filter
  raw.filter(low_freq, high_freq, fir_design='firwin', skip_by_annotation='edge')

    # event_train = eventsTrain(fp[f])
  epochs = mne.Epochs(raw, events=events, event_id=event_id, tmin=tmin, tmax=tmax, baseline=None, preload=True,
                        verbose=False)

  epochs_train = epochs.copy().crop(tmin=0.5, tmax=2.5)
  labels = epochs.events[:, -1]

    # # Define a monte-carlo cross-validation generator (reduce variance):
  epochs_data= epochs_train.get_data()
  return epochs_data,labels,mat_dic['test_idx']

intial_dic={}

total_length=280
train_num=10
test_num=10

for f,fla,cs in zip(fp,fplabel,pick_chan):
    x,y,testix=getdata_label(fp[f],fplabel[fla],pick_chan[cs])

    intial_dic[f]=[x,y]
    # # Assemble a classifier
    # 
    # csp = CSP(n_components=len(epochs.ch_names), reg=None, log=True, norm_trace=False)





Creating RawArray with float64 data, n_channels=18, n_times=298458
    Range : 0 ... 298457 =      0.000 ...  2984.570 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 7 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 165 samples (1.650 sec)

Creating RawArray with float64 data, n_channels=18, n_times=283574
    Range : 0 ... 283573 =      0.000 ...  2835.730 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 7 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal 

#Model construction

In [5]:
class CenterLossLayer(Layer):

    def __init__(self, alpha=0.5, **kwargs):
        super().__init__(**kwargs)
        self.alpha = alpha

    def build(self, input_shape):
        self.centers = self.add_weight(name='centers',
                                       shape=(2, 24),
                                       initializer='uniform',
                                       trainable=False)

        super().build(input_shape)

    def call(self, x, mask=None):

        # x[0] is Nx2, x[1] is Nx10 onehot, self.centers is 10x2
        delta_centers = K.dot(K.transpose(x[1]), (K.dot(x[1], self.centers) - x[0]))  # 10x2
        center_counts = K.sum(K.transpose(x[1]), axis=1, keepdims=True) + 1  # 10x1
        delta_centers /= center_counts
        new_centers = self.centers - self.alpha * delta_centers
        # self.add_update((self.centers, new_centers), x)
        
        self.centers.assign(new_centers)


        # self.add_update((self.counter, self.counter + 1), x)

        self.result = x[0] - K.dot(x[1], self.centers)
        # self.result = K.sum(self.result ** 2, axis=1, keepdims=True)#/ K.dot(x[1], center_counts)
        self.result = K.mean(self.result ** 2, axis=1, keepdims=True)
        return self.result # Nx1

    def compute_output_shape(self, input_shape):
        return K.int_shape(self.result)

    def get_config(self):
      cfg = super().get_config()
      return cfg  

### custom loss
def zero_loss(y_true, y_pred):
    # return 0.5 * K.mean(y_pred, axis=0)
    return 0.5 * K.mean(y_pred)



In [6]:
def EEGNet(F1=8,kernLength=16,D=2,Chans=18,dropout=0,regRate=.25,averpool_1=8,averpool_2=8):
    input1 = Input(shape = (1,Chans, 201))
    input2 = Permute((3,2,1))(input1)
    
    block1= Conv2D(F1, (kernLength, 1), padding = 'same',data_format='channels_last',use_bias = False)(input2)
    block1 = BatchNormalization(axis = -1)(block1)

    block2 = DepthwiseConv2D((1, Chans), use_bias = False, 
                                    depth_multiplier = D,
                                    data_format='channels_last',
                                    depthwise_constraint = max_norm(1.))(block1)
    block2 = BatchNormalization(axis = -1)(block2)
    block2 = Activation('elu')(block2)
    block2 = AveragePooling2D((averpool_1,1),data_format='channels_last')(block2)
    block2 = Dropout(dropout)(block2)

    block3 = SeparableConv2D(F1, (int(kernLength/2), 1),
                            data_format='channels_last',
                            use_bias = False, padding = 'same')(block2)
    block3 = BatchNormalization(axis = -1)(block3)        

    block3 = Activation('elu')(block3)
    block3 = AveragePooling2D((averpool_2,1),data_format='channels_last')(block3)
    block3 = Dropout(dropout)(block3)
    block3=Flatten()(block3)

    
    side = Activation(None, name = 'mmd')(block3)
    aux_input = Input((2,))
    center_side=CenterLossLayer(alpha=0.5, name='centerlosslayer')([block3, aux_input])


    dense = Dense(2, name = 'fe',kernel_constraint = max_norm(regRate))(block3)
    main = Activation('softmax', name = 'softmax')(dense)

    return Model([input1,aux_input], outputs=[main,side,center_side])

In [7]:
model=EEGNet()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1, 18, 201)  0           []                               
                                ]                                                                 
                                                                                                  
 permute (Permute)              (None, 201, 18, 1)   0           ['input_1[0][0]']                
                                                                                                  
 conv2d (Conv2D)                (None, 201, 18, 8)   128         ['permute[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 201, 18, 8)  32          ['conv2d[0][0]']             

3.1. mmd construct 

In [8]:
import numpy as np
from matplotlib import pyplot as plt
import math, os
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Conv2D, Conv2DTranspose, Dense, Input, Reshape, Flatten
from keras.models import Model
from keras.utils.vis_utils import plot_model
import pdb
from keras import backend as K
from keras.datasets import mnist
'MMD functions'
def compute_kernel(x, y):
    x_size = K.shape(x)[0]
    y_size = K.shape(y)[0]
    dim = K.shape(x)[1]
    tiled_x = K.tile(K.reshape(x, [x_size, 1, dim]), [1, y_size, 1])
    tiled_y = K.tile(K.reshape(y, [1, y_size, dim]), [x_size, 1, 1])
    return K.exp(-K.mean(K.square(tiled_x - tiled_y), axis=2) / K.cast(dim, 'float32'))

def compute_mmd(x, y):
    x_kernel = compute_kernel(x, x)
    y_kernel = compute_kernel(y, y)
    xy_kernel = compute_kernel(x, y)
    return K.mean(x_kernel) + K.mean(y_kernel) - 2 * K.mean(xy_kernel)

def custom_loss(source_or_target, y_pred):
    """
    Critical loss builder
		:param train_z: latent code
		:param train_xr: reconstructed data
		:param train_x: training data, the input data
		:return: new loss
		"""
    'So, we first get the mmd loss'
    'First, sample from random noise'
    source_or_target=tf.reshape(source_or_target,[-1])
    TF_source = tf.reshape(tf.where(tf.equal(source_or_target, 0)),[-1])
    TF_target = tf.reshape(tf.where(tf.equal(source_or_target, 1)),[-1])
    Source_fea=tf.gather(y_pred, TF_source)
    # batch_size = K.shape(y_pred)[0]
    # latent_dim = K.int_shape(y_pred)[1]
    # true_samples = K.random_normal(shape=(batch_size, latent_dim), mean=10., stddev=1.)
    Target_fea=tf.gather(y_pred, TF_target)
    
    
    'calculate mmd loss'

    loss_mmd = compute_mmd(Target_fea, Source_fea)

    'Then, also get the reconstructed loss'
    # loss_nll =  = K.categorical_crossentropy(y_true, y_pred)

    'Add them together, then you can get the final loss'
    loss =  loss_mmd*24
    return loss

3.2. Model complie

In [9]:
def model_complie(mdd_weight,center_weight):
  opt = Adam(learning_rate=0.001)
  model=EEGNet()
  model.compile(optimizer=opt, loss=["categorical_crossentropy",custom_loss,zero_loss],metrics=['accuracy'],loss_weights=[1,mdd_weight,center_weight])
  return model


In [10]:
from sklearn.preprocessing import OneHotEncoder


dic=[]
str_list=['aa','al','av','aw','ay']


for i in str_list:
  temp=[]
  x,y=intial_dic[i]
  x=x.reshape(x.shape[0],1,x.shape[1],-1)

  y=y-1

  onehot_encoder = OneHotEncoder(sparse=False)
  temp_y = y.reshape(len(y), 1)
  onehot_encoded = onehot_encoder.fit_transform(temp_y)
  temp.append(x)
  temp.append(y)
  temp.append(onehot_encoded)

  dic.append(temp)

3.4. Model_training

In [11]:


def mini_set(label_set,num):
  index1=np.where(label_set ==0)[0]
  index2=np.where(label_set == 1)[0]
  number=max(index1[num-1],index2[num-1])+1
  return number
  

In [12]:

def model_training(epochs,model,Subject_number):
  lsee=[]
  simple_model=model

  total_x,y,one_hot_y=dic[Subject_number][0],dic[Subject_number][1],dic[Subject_number][2]
  hyperparameter=10


  selecty=y

  mini_supportnum=mini_set(selecty,hyperparameter)

  total_x=total_x
  total_y=one_hot_y

  index1=np.where(selecty[:mini_supportnum] == 0)[0]
  index2=np.where(selecty[:mini_supportnum] == 1)[0]
  choice1=np.random.choice(index1,hyperparameter,replace=False)
  choice2=np.random.choice(index2,hyperparameter,replace=False)
  choice_total=np.hstack((choice1,choice2))
  test_list=list(range(mini_supportnum))
  xxxtest= np.delete(total_x, test_list,0)
  yyytest=np.delete(total_y, test_list,0)


  
  target_x=total_x[choice_total]
  target_y=total_y[choice_total]

  total_x_train=[]
  total_y_train=[]
  full_subjectnum=[0,1,2,3,4]
  full_subjectnum.remove(Subject_number)
  for train_subject in full_subjectnum:
    total_x,y,one_y=dic[train_subject][0],dic[train_subject][1],dic[train_subject][2]

    if len(total_x_train)==0:
      total_x_train=total_x
      total_y_train=one_hot_y
    else:
      total_x_train=np.vstack((total_x_train,total_x))
      total_y_train=np.vstack((total_y_train,one_hot_y))

  x_train=total_x_train
  y_train=total_y_train

  train_xx=np.vstack((x_train,target_x))
  train_yy=np.vstack((y_train,target_y))


  test_ones=np.ones((len(target_x)))
  test_zeros=np.zeros((len(x_train)))
  test_domian_label=np.hstack((test_ones,test_zeros))



  x_test_target=xxxtest
  y_test_onehot_target=yyytest






  checkpoint = ModelCheckpoint('model_best_'+str(i)+'.h5', verbose=1, monitor='val_softmax_accuracy',save_best_only=True, mode='max')  


  valid_transform=np.ones((len(x_test_target)))
  dummy1 = np.zeros((train_xx.shape[0], 1))
  dummy2 = np.zeros((x_test_target.shape[0], 1))

  simple_model.fit([train_xx,train_yy],[train_yy,test_domian_label,dummy1],batch_size=64, epochs=epochs, callbacks=[checkpoint],
          verbose=0, validation_data=([x_test_target,y_test_onehot_target], [y_test_onehot_target,valid_transform,dummy2]))



In [13]:
epochs=500
mdd_weight=0.1
center_weight=0.01
Subject_number=2
model=model_complie(mdd_weight,center_weight)

model_training(epochs,model,Subject_number)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/custom_loss/Reshape_25:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/custom_loss/Reshape_24:0", shape=(None, 24), dtype=float32), dense_shape=Tensor("gradient_tape/custom_loss/Cast_6:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/custom_loss/Reshape_27:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/custom_loss/Reshape_26:0", shape=(None, 24), dtype=float32), dense_shape=Tensor("gradient_tape/custom_loss/Cast_7:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This


Epoch 1: val_softmax_accuracy improved from -inf to 0.48263, saving model to model_best_ay.h5

Epoch 2: val_softmax_accuracy improved from 0.48263 to 0.49807, saving model to model_best_ay.h5

Epoch 3: val_softmax_accuracy did not improve from 0.49807

Epoch 4: val_softmax_accuracy did not improve from 0.49807

Epoch 5: val_softmax_accuracy did not improve from 0.49807

Epoch 6: val_softmax_accuracy did not improve from 0.49807

Epoch 7: val_softmax_accuracy did not improve from 0.49807

Epoch 8: val_softmax_accuracy did not improve from 0.49807

Epoch 9: val_softmax_accuracy did not improve from 0.49807

Epoch 10: val_softmax_accuracy did not improve from 0.49807

Epoch 11: val_softmax_accuracy did not improve from 0.49807

Epoch 12: val_softmax_accuracy improved from 0.49807 to 0.50965, saving model to model_best_ay.h5

Epoch 13: val_softmax_accuracy improved from 0.50965 to 0.53282, saving model to model_best_ay.h5

Epoch 14: val_softmax_accuracy improved from 0.53282 to 0.54054, s